In [1]:
# Drive Draft File Inspector
# Use this script to diagnose missing or broken files in your Google Drive.
# It scans the newest draft folder and lists all files, including trashed ones,
# which are invisible in the normal Drive view but can cause issues in other scripts.
#
# What it does:
#   1. Connects to Google Drive using OAuth credentials
#   2. Finds the newest Season folder
#   3. Navigates to the Pictures subfolder within that season
#   4. Finds the newest Draft folder (by date)
#   5. Lists all files, flagging any marked as [TRASHED]

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os
import re

# Configuration
from config import SCOPES, MAIN_FOLDER_ID

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
else:
    flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
    creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('drive', 'v3', credentials=creds)

print("Step 1: Looking for season folders...")
results = service.files().list(
    q=f"'{MAIN_FOLDER_ID}' in parents and mimeType='application/vnd.google-apps.folder'",
    fields="files(id, name)"
).execute()
folders = results.get('files', [])

max_season_num = 0
newest_season_id = None
newest_season_name = None

for folder in folders:
    match = re.search(r'Season (\d+)', folder['name'])
    if match:
        season_num = int(match.group(1))
        print(f"  Found: {folder['name']}")
        if season_num > max_season_num:
            max_season_num = season_num
            newest_season_id = folder['id']
            newest_season_name = folder['name']

assert newest_season_id is not None, "No season folders found in the main folder"
print(f"\nStep 2: Newest season is {newest_season_name}")

print(f"\nStep 3: Looking inside {newest_season_name}...")
results = service.files().list(
    q=f"'{newest_season_id}' in parents and mimeType='application/vnd.google-apps.folder'",
    fields="files(id, name)"
).execute()
season_folders = results.get('files', [])

pictures_folder_id = None
for folder in season_folders:
    print(f"  Found folder: {folder['name']}")
    if folder['name'].lower() == 'pictures':
        pictures_folder_id = folder['id']
        print(f"\nStep 4: Found Pictures folder!")

assert pictures_folder_id is not None, f"No 'Pictures' folder found inside {newest_season_name}"

print(f"\nStep 5: Looking for Draft folders in Pictures...")
results = service.files().list(
    q=f"'{pictures_folder_id}' in parents and mimeType='application/vnd.google-apps.folder'",
    fields="files(id, name)"
).execute()
picture_folders = results.get('files', [])

newest_draft_date = 0
newest_draft_id = None
newest_draft_name = None

for folder in picture_folders:
    match = re.match(r'(\d{8})\s+Draft\s+(\d+)', folder['name'])
    if match:
        date_num = int(match.group(1))
        print(f"  Found: {folder['name']} (date: {date_num})")

        if date_num > newest_draft_date:
            newest_draft_date = date_num
            newest_draft_id = folder['id']
            newest_draft_name = folder['name']

assert newest_draft_id is not None, "No draft folders found inside Pictures"
print(f"\nStep 6: Newest draft is {newest_draft_name}")

# Get MORE info about files including trashed status
print(f"\nStep 7: Getting detailed file info from {newest_draft_name}...")
results = service.files().list(
    q=f"'{newest_draft_id}' in parents",
    fields="files(id, name, trashed, createdTime)"
).execute()
files = results.get('files', [])

files_sorted = sorted(files, key=lambda x: x['name'])

print(f"\nFiles in {newest_draft_name}:")
for file in files_sorted:
    trashed_status = " [TRASHED]" if file.get('trashed', False) else ""
    print(f"  - {file['name']}{trashed_status}")

print(f"\nTotal files: {len(files)}")

Step 1: Looking for season folders...
  Found: Season 5
  Found: Season 4
  Found: Season 3
  Found: Season 2
  Found: Season 1

Step 2: Newest season is Season 5

Step 3: Looking inside Season 5...
  Found folder: Backups
  Found folder: Manacore
  Found folder: Pictures

Step 4: Found Pictures folder!

Step 5: Looking for Draft folders in Pictures...
  Found: 20260222 Draft 2 (date: 20260222)
  Found: 20260125 Draft 7 (date: 20260125)
  Found: 20251228 Draft 6 (date: 20251228)
  Found: 20251221 Draft 5 (date: 20251221)
  Found: 20250817 Draft 1 (date: 20250817)
  Found: 20251108 Draft 4 (date: 20251108)
  Found: 20251026 Draft 3 (date: 20251026)
  Found: 20250921 Draft 2 (date: 20250921)

Step 6: Newest draft is 20260222 Draft 2

Step 7: Getting detailed file info from 20260222 Draft 2...

Files in 20260222 Draft 2:

Total files: 0
